# work log - database disagreement evaluation

# Table of Contents

- [Setup](#Setup)

    - [Setup - Imports](#Setup---Imports)
    - [Setup - Initialize Django](#Setup---Initialize-Django)
    - [Setup - Database](#Setup---Database)

- [Work](#Work)
- [TODO](#TODO)

# Setup

- Back to [Table of Contents](#Table-of-Contents)

## Setup - Imports

- Back to [Table of Contents](#Table-of-Contents)

In [ ]:
# imports
import csv
import datetime
import json
import psycopg2
import psycopg2.extras
import six

print( "packages imported at " + str( datetime.datetime.now() ) )

In [ ]:
%pwd

In [ ]:
%ls

## Setup - Initialize Django

- Back to [Table of Contents](#Table-of-Contents)

First, initialize my dev django project, so I can run code in this notebook that references my django models and can talk to the database using my project's settings.

You need to have installed your virtualenv with django as a kernel, then select that kernel for this notebook.

In [ ]:
%run django_init.py

Import any `sourcenet` or `sourcenet_analysis` models or classes.

In [ ]:
# sourcenet_analysis models.
from sourcenet_analysis.models import Reliability_Names
from sourcenet_analysis.models import Reliability_Names_Evaluation

print( "sourcenet and sourcenet_analysis packages imported at " + str( datetime.datetime.now() ) )

## Setup - Database

- Return to [Table of Contents](#Table-of-Contents)

In [ ]:
# ==> PostgreSQL (psycopg2) example:
db_type = "postgresql"
db_username = "<username>"
db_password = "<password>"
db_host = "research.local"
db_port = "5432"
db_database = "sourcenet"
db_encoding = "utf8"

# create psycopg2 connection to PostgreSQL using connection variables defined above.
pgsql_connection = psycopg2.connect( host = db_host, port = db_port, database = db_database, user = db_username, password = db_password )

print( "psycopg2 database connection created at " + str( datetime.datetime.now() ) + ": " + str( pgsql_connection ) )

In [ ]:
# create psycopg2 cursor using pgsql_connection.
pgsql_cursor = pgsql_connection.cursor( cursor_factory = psycopg2.extras.DictCursor )

print( "psycopg2 database cursor created at " + str( datetime.datetime.now() ) + ": " + str( pgsql_cursor ) )

In [ ]:
# rollback transaction:
pgsql_connection.rollback()

print( "connection transaction rolled back at " + str( datetime.datetime.now() ) )

# Work

- Back to [Table of contents](#Table-of-contents)

Read and parse delimited file of Reliability_Names rows we've arleady processed.  Create database row for each.

In [ ]:
# CONSTANTS-ish

# header indexes:
#ID | Name | Article | Article_Data_List | Status | Error? (SHB = Should Have Been)| Notes
INDEX_RELIABILITY_NAMES_ID = 0
INDEX_PERSON_NAME = 1
INDEX_ARTICLE_INFO = 2
INDEX_ARTICLE_DATA_LIST = 3
INDEX_STATUS = 4
INDEX_STATUS_MESSAGE = 5
INDEX_NOTES = 6

# declare variables
delimited_file_path = ""
delimited_file = None
delimited_reader = None
current_row = None
rn_instance = None
rne_instance = None
row_counter = -1
manual_counter = -1

# declare variables - column values
reliability_names_id = -1
person_name = ""
article_info = ""
article_data_info = ""
status = ""
status_message = ""
notes = ""
label = ""

# declare varibles - create by hand
work_string = ""
work_list = []
article_id = -1
article_data_list = []
article_data_item = None
article_data_id_list = []
article_data_id = -1

# file path?
delimited_file_path = "reliability_names_evaluations.txt"

# read file
with open( delimited_file_path ) as delimited_file:
    
    # CSV reader
    delimited_reader = csv.reader( delimited_file, delimiter='|' )

    # first row is column names
    current_row = six.next( delimited_reader )
    
    # init shared values
    label = "prelim_month"
    
    # now loop over rest of rows.
    row_counter = 0
    manual_counter = 0
    for current_row in delimited_reader:
        
        # increment row counter
        row_counter += 1
        
        # print out the row.
        column_count = len( current_row )
        print( "==> row column count = " + str( column_count ) + " ( " + str( column_count == 7 ) + " ) - row: " + ",".join( current_row ) )
        
        # get values
        reliability_names_id = current_row[ INDEX_RELIABILITY_NAMES_ID ].strip()
        reliability_names_id = int( reliability_names_id )
        person_name = current_row[ INDEX_PERSON_NAME ].strip()
        article_info = current_row[ INDEX_ARTICLE_INFO ].strip()
        article_data_info = current_row[ INDEX_ARTICLE_DATA_LIST ].strip()
        status = current_row[ INDEX_STATUS ].strip()
        status_message = current_row[ INDEX_STATUS_MESSAGE ].strip()
        notes = current_row[ INDEX_NOTES ].strip()
        
        # parse out additional data.
        
        # ==> article_id
        # - Article [20645](http://research.local/sourcenet/sourcenet/article/article_data/view_with_text/?article_id=20645)

        # split first on the leading square bracket around the ID.
        work_list = article_info.split( "[" )

        # then split item 2 (index 1) on right square bracket.
        work_string = work_list[ 1 ]
        work_list = work_string.split( "]" )
        article_id = work_list[ 0 ].strip()
        article_id = int( article_id )
        print( "----> article_id = " + str( article_id ) )

        # ==> article_data_id
        # - Article_Data: [2322 (coder=8)](http://research.local/sourcenet/sourcenet/article/article_data/view/?article_id=20645&article_data_id_select=2322); [2984 (coder=2)](http://research.local/sourcenet/sourcenet/article/article_data/view/?article_id=20645&article_data_id_select=2984)

        # split first on semi-colon
        article_data_list = article_data_info.split( ";" )
        article_data_id_list = []
        for article_data_item in article_data_list:

            # use work_list to isolate the ID - first, split on left square bracket.
            work_list = article_data_item.split( "[" )

            # get 2nd thing in list.
            work_string = work_list[ 1 ]

            # split then on left paren
            work_list = work_string.split( "(" )

            # get the first item in list.
            article_data_id = work_list[ 0 ].strip()
            article_data_id = int( article_data_id )

            # add to list.
            article_data_id_list.append( article_data_id )

            print( "----> article_data_id = " + str( article_data_id ) )

        #-- END loop over article data items. --#
        
        print( "----> status = " + status )
        
        # call Reliability_Names_Evaluation.create_from_reliability_data()
        rne_instance = Reliability_Names_Evaluation.create_from_reliability_data(
                           reliability_names_id,
                           label_IN = label,
                           person_name_IN = person_name,
                           article_id_IN = article_id,
                           article_data_id_list_IN = article_data_id_list,
                           status_IN = status,
                           status_message_IN = status_message,
                           notes_IN = notes
                       )
        
    #-- END loop over rows. --#
    
#-- END with open( delimited_file_path ) --#

In [ ]:
# DROP all rows from the table.
table_name = "sourcenet_analysis_reliability_names_evaluation"
#sql_string = "DELETE FROM " + table_name
#sql_string += ";"

# run it.
#pgsql_cursor.execute( sql_string )

# loop over records in table.
evaluation_qs = None
evaluation_qs = Reliability_Names_Evaluation.objects.all()
eval_record = None
record_count = 0
for eval_record in evaluation_qs:
    
    # counter
    record_count += 1
    
    # output row details:
    print( eval_record )
    
    # delete.
    eval_record.delete()
    
#-- END loop over records. --#

print( "deleted " + str( record_count ) + " records." )

# reset the unique ID sequences to 0.
sequence_name_list = []
sequence_name_list.append( table_name + "_id_seq" )
sequence_name_list.append( "sourcenet_analysis_reliability_names_evaluation_article__id_seq" )
sequence_name_list.append( "sourcenet_analysis_reliability_names_evaluation_persons_id_seq" )

for sequence_name in sequence_name_list:
    
    sql_string = "ALTER SEQUENCE " + sequence_name + " RESTART WITH 1;"
    print( "==> " + sql_string )
    pgsql_cursor.execute( sql_string )
    
#-- END loop over sequence names to reset. --#

# commit
pgsql_connection.commit()

print( "reset table " + table_name + " at " + str( datetime.datetime.now() ) )

# TODO

- Back to [Table of Contents](#Table-of-Contents)

TODO:

- // check if Reliability_Names exists.  If not, parse Article, Article_Data, Persons, and make a row by hand...
- // wipe table and re-run.
- // go through and update by hand the flags for the other lists.
- // make a list of those that were merged.
- update with single name removals from "`remove_single_names-work_log.ipynb`"